In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import html
import re
import os
from bs4 import BeautifulSoup
%matplotlib inline
from tidylib import tidy_fragment

In [2]:
post_df = pd.read_csv('ssu_posts.csv', index_col=0)

/home/aaron/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
fixed = post_df.head(20).text.apply(tidy_fragment).str.get(0)

In [4]:
fixed.values

array(['Final Test', '^_^', 'Hey everyone!', 'Hey Cecil.', "What's up?",
       'Test.', 'Theeeeeeeeere we go.', "Okay, it's all good now. :p",
       'Test.', 'Gooooody.', 'Hmmm....', 'Testing again.',
       'Pretty much my last test.',
       "Everything's satisfactory. Transition back to this chat complete. ^_^",
       'BBL, in a few hours.', 'yay ^_^', 'LOL@Picture Bug',
       'Test without picture', 'with pics', 'again'], dtype=object)

In [5]:
post_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1711195 entries, 0 to 1711192
Data columns (total 3 columns):
handle        object
text          object
time_stamp    object
dtypes: object(3)
memory usage: 52.2+ MB


In [6]:
post_df.text.fillna('', inplace=True)

In [7]:
post_df['text_parsed'] = post_df['text'].apply(tidy_fragment).str.get(0)

In [8]:
post_df['text_parsed'] = post_df['text_parsed'].apply(html.unescape)

In [ ]:
posts = post_df.text_parsed.values

In [ ]:
img_mask = post_df.text_parsed.str.contains('<img')

In [ ]:
img_mask.sum()

In [ ]:
img_mask.sum()/len(post_df)

In [9]:
EMOTICON_MAP = {
    '^_^': '[animesmile]',
    '~_^': '[animewink]',
    '^_~': '[animewink]',
    '^_-': '[animewink]',
    '-_^': '[animewink]',
    ':p': '[tongue]',
    ':P': '[tongue]',
    ';p': '[tonguewink]',
    ';P': '[tonguewink]',
    ':)': '[smile]',
    ';)': '[wink]',
    ':-)': '[smile]',
}

In [10]:
url_match = re.compile('https?:\/\/.*[\r\n]*')

In [14]:
def parse_chat_posts(post, emoticon_map=EMOTICON_MAP, url_match=url_match):
    if '<' in post:
        post_soup = BeautifulSoup(post, "lxml")
        for img_tag in post_soup.findAll('img'):
            alt = img_tag.get('alt')
            if alt is not None:
                img_tag.replace_with(alt.lower())
                continue

            src = img_tag.get('src')
            if src is None:
                img_tag.replace_with('[unknown]')
                continue
            file_name = src.split('/')[-1]
            file_name = file_name.split('.')[0]
            img_tag.replace_with('[{}]'.format(file_name.lower()))

        bs_parsed_text = post_soup.text
    else:
        bs_parsed_text = post
    
    for emoticon, text in emoticon_map.items():
        bs_parsed_text = bs_parsed_text.replace(emoticon, text)
    
    if url_match is not None:
        bs_parsed_text = url_match.sub('[url]', bs_parsed_text)

    return bs_parsed_text

In [15]:
post_df['text_parsed_bs'] = post_df.text_parsed.apply(parse_chat_posts)

In [16]:
post_df['text_parsed_bs'].tail(20).values

array([':o', '-- :',
       '2010-12-17 15:21:30 Zenkci : Let me ask everyone a question: It is your first few days/weeks at a new job and you are training with a veteran employee. The work day can and may be slow at times; do you use this time to ask questions or play with your smart phone while chatting on Facebook? Or, on that same note the trainer is telling something, you get a text message, do you check while they are still taking to you?',
       '(brandon): HI HERO', 'Stalker.', '(nmartinez): Hey Darrow!',
       'Testing something', '[firepit][hero]',
       'You broke my avatar, you jerk.', 'So broken!',
       "Hey DP; if you happen to see this. We've moved to slack. [url]",
       '(nmartinez): Testing?', 'STOP SPYING ON ME', 'I am over here!',
       '(nmartinez): You!', '(brandon): NO YOU!',
       '(aawiegel): Calm down kiddos!', '[url]',
       'Yeah. But Joe will Make Slack Great Again.', '/me waves.'], dtype=object)

In [17]:
post_df.handle.fillna('', inplace=True)

In [18]:
post_df['handle'] = post_df.handle.str.split(' on:').str.get(0)

In [19]:
post_df['handle'] = post_df.handle.str.split(' From:').str.get(0)

In [20]:
post_df['handle'].value_counts()

Janus                                                 287940
Nadia                                                 240360
Vavalion                                              127439
Psi-Amy                                                85779
Darrow                                                 80400
Armadillo                                              76683
Red Leader                                             73382
Aaron                                                  72906
Hero                                                   70740
DP                                                     68988
Dark Paladin                                           66132
Cecil                                                  62365
Randofu                                                56251
Jester                                                 18492
Driretlan                                              18167
Piki                                                   16988
Soul                    

In [21]:
post_df['handle'] = post_df.handle.apply(tidy_fragment).str.get(0)

In [23]:
post_df['handle'] = post_df.handle.apply(html.unescape)

In [24]:
post_df['handle'] = post_df.handle.apply(parse_chat_posts, args=(EMOTICON_MAP, None))

In [25]:
handle_map = {'DP': 'Dark Paladin',
              'Lady Sraphim': 'Sraphene',
              'NickTUH': 'Hero',
              'J': 'Jester',
              'Psiren': 'Psi-Amy',
              'Randofuzz0rx': 'Randofu',
              'Moggle': 'Mog',
              'Uber Ruser': 'Randofu',
              'K\'van': 'K\'van Cetre',
              'Zenkci': 'Hero',
              'Phosphene': 'Sraphene',
              'Sraphim': 'Sraphene',
              'Ruser': 'Randofu',
              'Sraphy': 'Sraphene',
              'red leader': 'Red Leader'}

In [26]:
post_df['handle_fixed'] = post_df.handle.replace(handle_map)

In [28]:
post_df['handle'] = post_df['handle_fixed']

In [29]:
post_df.drop(columns='handle_fixed', inplace=True)

In [30]:
post_df.handle.value_counts()

Janus                                                 287940
Nadia                                                 240362
Dark Paladin                                          135120
Vavalion                                              127439
Psi-Amy                                                92295
Hero                                                   82878
Darrow                                                 80401
Armadillo                                              76683
Red Leader                                             74406
Aaron                                                  72906
Randofu                                                64866
Cecil                                                  62365
Sraphene                                               27703
Jester                                                 26622
Driretlan                                              18167
Piki                                                   16988
Soul                    

In [31]:
post_df.tail()

,handle,text,time_stamp,text_parsed,text_parsed_bs
1711188,SSU Slack Bot,(brandon): NO YOU!,2017-05-26 17:33:32,(brandon): NO YOU!,(brandon): NO YOU!
1711189,SSU Slack Bot,(aawiegel): Calm down kiddos!,2017-05-26 17:34:52,(aawiegel): Calm down kiddos!,(aawiegel): Calm down kiddos!
1711190,Hero,"<a href=""http://chat.ssuniverse.com/archives_l...",2017-05-26 17:35:43,"<a href=""http://chat.ssuniverse.com/archives_l...",[url]
1711191,Janus,Yeah. But Joe will Make Slack Great Again.,2017-06-19 15:26:40,Yeah. But Joe will Make Slack Great Again.,Yeah. But Joe will Make Slack Great Again.
1711192,Janus,/me waves.,2017-10-04 16:54:35,/me waves.,/me waves.


In [32]:
post_df.head()

,handle,text,time_stamp,text_parsed,text_parsed_bs
0,Hero,Final Test,2001-11-01 00:00:01,Final Test,Final Test
1,Asuka,^_^,2001-11-01 00:00:02,^_^,[animesmile]
2,Cecil,Hey everyone!,2001-11-01 00:00:03,Hey everyone!,Hey everyone!
3,No Name 1029,Hey Cecil.,2001-11-01 00:00:04,Hey Cecil.,Hey Cecil.
4,Cecil,What&#39;s up?,2001-11-01 00:00:05,What's up?,What's up?


In [33]:
post_df.to_csv('log_bs_parsed.csv', index=False)

In [34]:
post_df = pd.read_csv('log_bs_parsed.csv')

In [35]:
len(post_df.handle.unique())

2473

In [36]:
post_df.loc[post_df.handle.isnull(), 'handle'] = 'Null Handle'

In [37]:
post_df.handle.value_counts().describe()

count      2473.000000
mean        691.951072
std        9517.101209
min           1.000000
25%           1.000000
50%           3.000000
75%          25.000000
max      287940.000000
Name: handle, dtype: float64

In [38]:
GUESS_WHO_PATH = os.path.join(os.path.curdir, 'guess_who')

In [ ]:
post_df.shift(periods=-2).head(), post_df.head()

In [39]:
handle_df = post_df.loc[:, ['time_stamp', 'handle', 'text_parsed_bs']].rename(columns={'text_parsed_bs': 'text'})

In [40]:
ref_nums = [-2, -1, 1, 2]

In [41]:
combined_posts = handle_df.time_stamp + ' ' + handle_df.handle + ': ' + handle_df.text

In [42]:
for ref_num in ref_nums:
    if ref_num > 0:
        ref_str = 'back'
        
    else:
        ref_str = 'forward'
        
    handle_df['{}_{}_post'.format(ref_str, -1*ref_num)] = combined_posts.shift(periods=ref_num)

In [43]:
handle_df['guess_who'] = ''

In [44]:
handle_df = handle_df.loc[:, ['time_stamp', 'handle', 'back_-2_post', 
                              'back_-1_post', 'text', 'forward_1_post', 'forward_2_post', 'guess_who']]

In [45]:
handle_df.head()

,time_stamp,handle,back_-2_post,back_-1_post,text,forward_1_post,forward_2_post,guess_who
0,2001-11-01 00:00:01,Hero,NaN,NaN,Final Test,2001-11-01 00:00:02 Asuka: [animesmile],2001-11-01 00:00:03 Cecil: Hey everyone!,
1,2001-11-01 00:00:02,Asuka,NaN,2001-11-01 00:00:01 Hero: Final Test,[animesmile],2001-11-01 00:00:03 Cecil: Hey everyone!,2001-11-01 00:00:04 No Name 1029: Hey Cecil.,
2,2001-11-01 00:00:03,Cecil,2001-11-01 00:00:01 Hero: Final Test,2001-11-01 00:00:02 Asuka: [animesmile],Hey everyone!,2001-11-01 00:00:04 No Name 1029: Hey Cecil.,2001-11-01 00:00:05 Cecil: What's up?,
3,2001-11-01 00:00:04,No Name 1029,2001-11-01 00:00:02 Asuka: [animesmile],2001-11-01 00:00:03 Cecil: Hey everyone!,Hey Cecil.,2001-11-01 00:00:05 Cecil: What's up?,2001-11-01 00:00:06 Janus: Test.,
4,2001-11-01 00:00:05,Cecil,2001-11-01 00:00:03 Cecil: Hey everyone!,2001-11-01 00:00:04 No Name 1029: Hey Cecil.,What's up?,2001-11-01 00:00:06 Janus: Test.,2001-11-01 00:00:07 Janus: Theeeeeeeeere we go.,


In [46]:
handle_df[handle_df.handle.str.startswith('Red or')].to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_red_or_nadia.csv'), index=False)

In [47]:
orphan_handles1 = handle_df.handle.value_counts()[handle_df.handle.value_counts() == 1].index.tolist()

In [48]:
handle_df[handle_df.handle.isin(orphan_handles1)].to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge1.csv'), index=False)

In [49]:
handle_counts = handle_df.handle.value_counts()

In [50]:
handle_mask = (handle_counts >= 2) & (handle_counts <= 4)

In [51]:
handles_2_to_4 = handle_counts[handle_mask].index.tolist()

In [52]:
twofour_df = handle_df[handle_df.handle.isin(handles_2_to_4)].sort_values(['handle', 'time_stamp'])

In [53]:
twofour_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge2-4.csv'), index=False)

In [54]:
handle_mask = (handle_counts >= 5) & (handle_counts <= 6)

In [55]:
handles_5_to_6 = handle_counts[handle_mask].index.tolist()

In [56]:
fivesix_df = handle_df[handle_df.handle.isin(handles_5_to_6)].sort_values(['handle', 'time_stamp'])

In [57]:
fivesix_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge5-6.csv'), index=False)

In [58]:
handles_7_to_10_mask = (handle_counts >= 7) & (handle_counts <= 10)

In [59]:
handles_7_to_10 = handle_counts[handles_7_to_10_mask].index.tolist()

In [60]:
seventen_df = handle_df[handle_df.handle.isin(handles_7_to_10)].sort_values(['handle', 'time_stamp'])

In [61]:
seventen_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge7-10.csv'), index=False)

In [62]:
handles_11_to_20_mask = (handle_counts >= 11) & (handle_counts <= 20)

In [63]:
handles_11_to_20 = handle_counts[handles_11_to_20_mask].index.tolist()

In [64]:
eleventwenty_df = handle_df[handle_df.handle.isin(handles_11_to_20)].sort_values(['handle', 'time_stamp'])

In [65]:
eleventwenty_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge11-20.csv'), index=False)

In [66]:
handles_21_to_50_mask = (handle_counts >= 21) & (handle_counts <= 50)

In [67]:
handles_21_to_50 = handle_counts[handles_21_to_50_mask].index.tolist()

In [68]:
twentyonefifty_df = handle_df[handle_df.handle.isin(handles_21_to_50)].sort_values(['handle', 'time_stamp'])

In [69]:
twentyonefifty_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge21-50.csv'), index=False)

In [70]:
handles_51_to_100_mask = (handle_counts >= 51) & (handle_counts <= 100)

In [71]:
handles_51_to_100 = handle_counts[handles_51_to_100_mask].index.tolist()

In [72]:
fiftyhundred_df = handle_df[handle_df.handle.isin(handles_51_to_100)].sort_values(['handle', 'time_stamp'])

In [73]:
fiftyhundred_df.head()

,time_stamp,handle,back_-2_post,back_-1_post,text,forward_1_post,forward_2_post,guess_who
864769,2004-03-29 15:45:39,: Ranzera,2004-03-29 15:44:09 Nadia: Red: [siiiiiiiiiiii...,2004-03-29 15:44:24 Nadia: Red: Stuff.,what version are you using?,2004-03-29 15:47:03 Nadia: Red: I forgot alrea...,2004-03-29 15:48:06 : Ranzera: Oh! so you have...,
864771,2004-03-29 15:48:06,: Ranzera,2004-03-29 15:45:39 : Ranzera: what version ar...,2004-03-29 15:47:03 Nadia: Red: I forgot alrea...,Oh! so you haven't used it yet?,2004-03-29 15:48:53 Nadia: Red: Nope. And you ...,2004-03-29 15:49:44 : Ranzera: the name says a...,
864773,2004-03-29 15:49:44,: Ranzera,2004-03-29 15:48:06 : Ranzera: Oh! so you have...,2004-03-29 15:48:53 Nadia: Red: Nope. And you ...,the name says all. I would be Piki's Ex. I've ...,"2004-03-29 15:49:59 Nadia: Red: God, I feel li...",2004-03-29 15:50:08 Nadia: Ah.,
864776,2004-03-29 15:50:26,: Ranzera,"2004-03-29 15:49:59 Nadia: Red: God, I feel li...",2004-03-29 15:50:08 Nadia: Ah.,"dude, calm down. If computers are one thing, t...",2004-03-29 15:51:23 : Ranzera: I personally us...,2004-03-29 15:52:03 : Ranzera: you could also ...,
864777,2004-03-29 15:51:23,: Ranzera,2004-03-29 15:50:08 Nadia: Ah.,"2004-03-29 15:50:26 : Ranzera: dude, calm down...","I personally use a combination of Adaware, AVG...",2004-03-29 15:52:03 : Ranzera: you could also ...,2004-03-29 15:55:16 Randofu: Errr...? [feak],


In [74]:
fiftyhundred_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge51-100.csv'), index=False)

In [75]:
handles_101_to_1000_mask = (handle_counts >= 101) & (handle_counts <= 1000)

In [76]:
handles_101_to_1000 = handle_counts[handles_101_to_1000_mask].index.tolist()

In [77]:
hundredonethousand_df = handle_df[handle_df.handle.isin(handles_101_to_1000)].sort_values(['handle', 'time_stamp'])

In [78]:
hundredonethousand_df.to_csv(os.path.join(GUESS_WHO_PATH, 'guess_who_challenge101-1000.csv'), index=False)

In [79]:
handle_counts[handle_counts > 1000]

Janus                    287940
Nadia                    240362
Dark Paladin             135120
Vavalion                 127439
Psi-Amy                   92295
Hero                      82878
Darrow                    80401
Armadillo                 76683
Red Leader                74406
Aaron                     72906
Randofu                   64866
Cecil                     62365
Sraphene                  27703
Jester                    26622
Driretlan                 18167
Piki                      16988
Soul                      16433
NeueZiel                  13999
Becky                     12349
K'van Cetre                9775
evilbob                    9351
Mog                        8815
Random Quote               7150
Blaze                      6797
Null Handle                4412
Artemis                    4401
Red or Nadia               4233
Cascadius                  3555
Ilsyn                      2787
Illia                      2597
Hawk                       2329
DM      

In [80]:
handle_counts[handle_counts > 1000].to_csv('regulars.csv')

In [ ]:
fig, ax = plt.subplots()
handle_counts.value_counts().hist(bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1000], ax = ax)
ax.set_xlim([0, 11])

In [ ]:
post_df.handle.value_counts()[post_df.handle.value_counts() == 22]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

In [ ]:
def get_topical_terms(topic_mat, topic_num, term_vec, num_terms=25):
    return term_vec[np.argsort(-topic_mat[topic_num,:])][:num_terms]

In [ ]:
count_vec = CountVectorizer(min_df=20, stop_words='english')

In [ ]:
word_mat = count_vec.fit_transform(post_df['text_parsed_bs'])

In [ ]:
count_vec.get_feature_names()[:10]

In [ ]:
nmf = NMF(50)

In [ ]:
nmf_mat = nmf.fit_transform(word_mat)

In [ ]:
terms = np.array(count_vec.get_feature_names())
for i in range(10):
    print("topic {}".format(i))
    print(get_topical_terms(nmf.components_,i,terms, 50))

In [ ]:
terms = np.array(count_vec.get_feature_names())
for i in range(10, 20, 1):
    print("topic {}".format(i))
    print(get_topical_terms(nmf.components_,i,terms, 35))

In [ ]:
post_df['text_parsed_bs'].head(100).values

In [ ]:
post_df.to_csv('posts_bs_parsed.csv', index=False)

In [ ]:
post_df.text_parsed_bs.isnull().sum()/len(post_df)

In [ ]:
post_df.loc[img_mask, 'text_parsed'].head(20).values

In [ ]:
new_img_mask = post_df.text_parsed.str.contains('<img src="emoticons')

In [ ]:
new_img_mask.sum()

In [ ]:
post_df[new_img_mask].head()

In [ ]:
post_df[img_mask & ~new_img_mask].tail()

In [ ]:
post_df['time_stamp_parsed'] = pd.to_datetime(post_df.time_stamp)

In [ ]:
date_mask = post_df['time_stamp_parsed'] < pd.Timestamp('2005-01-24')

In [ ]:
post_df.loc[date_mask, 'text_parsed'].tail().values